In [1]:
import pandas as pd
from tqdm.auto import tqdm
import ast

tqdm.pandas()

In [2]:
df = pd.read_csv("./data/credits.csv")

In [3]:
df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [4]:
def move_movie_id_in_jsons(row: pd.Series) -> pd.Series:
    cast, crew, movie_id = (
        ast.literal_eval(row["cast"])[:5],
        ast.literal_eval(row["crew"])[:3],
        row["id"],
    )

    for c in cast:
        c["movie_id"] = movie_id

    for c in crew:
        c["movie_id"] = movie_id

    return pd.Series([cast, crew], index=["cast", "crew"])

In [5]:
df = df.progress_apply(move_movie_id_in_jsons, axis=1)

  0%|          | 0/45476 [00:00<?, ?it/s]

In [6]:
cast, crew = df["cast"], df["crew"]

In [7]:
cast = cast[cast.map(lambda x: len(x) > 0)].reset_index(drop=True)
crew = crew[crew.map(lambda x: len(x) > 0)].reset_index(drop=True)

In [8]:
cast_df = pd.DataFrame(cast.explode().reset_index(drop=True).values.tolist())

In [9]:
crew_df = pd.DataFrame(crew.explode().reset_index(drop=True).values.tolist())

In [10]:
cast_info = (
    cast_df[["id", "name", "gender"]]
    .drop_duplicates(subset=["id"])
    .reset_index(drop=True)
)
cast_movie_relationship = cast_df[["id", "movie_id", "character"]].rename(
    columns={"id": "actor_id"}
)

In [11]:
crew_info = (
    crew_df[["id", "name", "gender"]]
    .drop_duplicates(subset=["id"])
    .reset_index(drop=True)
)
crew_movie_relationship = crew_df[["id", "movie_id", "department"]].rename(
    columns={"id": "actor_id"}
)
crew_movie_relationship["department"] = crew_movie_relationship[
    "department"
].str.upper()

In [14]:
cast_info.to_csv("./data/cast_info.csv", index=False)
cast_movie_relationship.to_csv("./data/cast_movie_relationship.csv", index=False)
crew_info.to_csv("./data/crew_info.csv", index=False)
crew_movie_relationship.to_csv("./data/crew_movie_relationship.csv", index=False)

# Movies

In [3]:
m_df = pd.read_csv("./data/movies_metadata.csv")

/tmp/ipykernel_28830/3612021373.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  m_df = pd.read_csv("./data/movies_metadata.csv")


In [5]:
m_df.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [7]:
m_df["id"].nunique() == m_df.shape[0]

False

In [9]:
m_df["imdb_id"].nunique() == m_df.shape[0]

False

In [30]:
m_df = m_df[~m_df["id"].duplicated()]

In [63]:
import json

def load_and_dump_json(x: str) -> str:
    list_of_dicts = ast.literal_eval(x)
    return json.dumps(list_of_dicts)

In [ ]:
m_df["genres"] = m_df["genres"].map(load_and_dump_json)

In [69]:
m_df["genres"]

0        [{"id": 16, "name": "Animation"}, {"id": 35, "...
1        [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2        [{"id": 10749, "name": "Romance"}, {"id": 35, ...
3        [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...
4                           [{"id": 35, "name": "Comedy"}]
                               ...                        
45461    [{"id": 18, "name": "Drama"}, {"id": 10751, "n...
45462                        [{"id": 18, "name": "Drama"}]
45463    [{"id": 28, "name": "Action"}, {"id": 18, "nam...
45464                                                   []
45465                                                   []
Name: genres, Length: 45436, dtype: object

In [73]:
m_df = m_df[["id", "original_title", "budget", "original_language", "revenue", "runtime", "genres"]]

In [80]:
invalid_budget_rows = m_df[m_df["budget"].str.contains(".jpg")].index

In [ ]:
m_df.drop(invalid_budget_rows, axis=0, inplace=True)

In [85]:
m_df.head()

,id,original_title,budget,original_language,revenue,runtime,genres
0,862,Toy Story,30000000,en,373554033.0,81.0,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 35, ""..."
1,8844,Jumanji,65000000,en,262797249.0,104.0,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""..."
2,15602,Grumpier Old Men,0,en,0.0,101.0,"[{""id"": 10749, ""name"": ""Romance""}, {""id"": 35, ..."
3,31357,Waiting to Exhale,16000000,en,81452156.0,127.0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam..."
4,11862,Father of the Bride Part II,0,en,76578911.0,106.0,"[{""id"": 35, ""name"": ""Comedy""}]"


In [86]:
m_df.to_csv("./data/movies.csv", index=False)